In [ ]:
from utz import *
from njdot import crashes, occupants, vehicles, pedestrians, drivers
from njdot.crashes import Crashes, ksi_renames, road_cols
from njdot.data import cc2cn, cn2cc
from njdot.load import pk_base
from njdot.paths import CNS, CRASHES_DB
from nj_crashes.paths import PUBLIC_DIR

In [ ]:
%%time
c = crashes.load()
c

In [ ]:
c.loc[c.index.to_series().duplicated(keep=False)]

In [ ]:
pk_base

In [ ]:
c[c.duplicated(keep=False, subset=pk_base)]

In [ ]:
s = c[['year','cc', 'mc', 'case']]
s.equals(s.sort_values(s.columns.tolist()))

In [ ]:
ptc = c[(c.cc == 11) & (c.mc == 14) & (c.tk > 0)]
ptc

In [ ]:
o = occupants.load()
o

In [ ]:
o[o.crash_id.isin(ptc.index)]

In [ ]:
c[c.duplicated(keep=False, subset=pk_cols)]

In [ ]:
type(c)

In [ ]:
assert c[['cc', 'cn']].set_index('cc')['cn'].groupby(lambda x:x).apply(lambda s: singleton(s.tolist())).to_dict() == cc2cn

In [ ]:
gdf = c.gdf('o')
gdf

### Load county geometries

In [ ]:
from nj_crashes.geo import get_boundary_lls, get_county_coords, get_counties, get_county_geometries
from geopandas.tools import sjoin
import geopandas as gpd
from shapely import box
import simplejson
from njdot.codes import CrashSeverity

In [ ]:
%%time
cg = get_county_geometries()
cg

In [ ]:
%%time
joined = sjoin(gdf.df[['olat', 'olon', 'geometry']], cg)
joined = joined.rename(columns={ 'index_right': 'ocn', })
joined

In [ ]:
joined.ocn.value_counts().sort_index()

In [ ]:
%%time
with_ocn = sxs(gdf.df, joined.ocn).sort_index()

In [ ]:
%%time
occ = with_ocn.ocn.map(cn2cc).astype('Int64').rename('occ')
with_ocn = sxs(with_ocn, occ)
with_ocn

In [ ]:
assert with_ocn.index.tolist() == list(range(len(with_ocn)))

In [ ]:
ill = Crashes(with_ocn).mp_lls(append=True)
ill

In [ ]:
%%time
igdf = Crashes(ill).gdf('i')
igdf

In [ ]:
type(igdf)

In [ ]:
%%time
ij = sjoin(igdf.df[['geometry']], cg)
ij = ij.rename(columns={ 'index_right': 'icn', })
ij

In [ ]:
%%time
with_cns = sxs(igdf.df, ij.icn).sort_index()
icc = with_cns.icn.map(cn2cc).astype('Int64').rename('icc')
with_cns = sxs(with_cns, icc)
with_cns

In [ ]:
with_cns.groupby(['cc', 'mc'])['mn'].apply(lambda s: s.value_counts())

In [ ]:
assert with_cns.index.tolist() == list(range(len(with_cns)))

In [ ]:
type(with_cns)

In [ ]:
ill_cols = ['icc', 'ilat', 'ilon']
oll_cols = ['occ', 'olat', 'olon']
sql_cols = [
    'dt',
    *pk_cols,
    *ksi_renames.values(),
    'crash_type', 'severity',
    *road_cols,
    *ill_cols,
    *oll_cols,
]

In [ ]:
sd = with_cns[sql_cols]
sd

In [ ]:
from nj_crashes.utils import sql

def write_sql(df, db_path, rm=True, tbl='crashes'):
    return sql.write(
        df, tbl=tbl, db_path=db_path,
        idxs=[
            ('severity', 'dt', 'cc', 'mc'),
            ('severity', 'ilat', 'ilon'),
            ('severity', 'icc', 'dt'),
        ]
    )

In [ ]:
%%time
cur = write_sql(sd, CRASHES_DB)
cur

In [ ]:
cur.execute("Select * from SQLite_master")
idxs = cur.fetchall()
idxs

In [ ]:
%%time
with_cns.drop(columns='geometry').to_parquet(CNS)